In [1]:
import PySimpleGUI as sg
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2
import horizon
import control
import dynamics
import plotter3D
import plotter2D

# Horizon Image
horiz_base = cv2.imread("images/horizon_base.png")
horiz_outer = cv2.imread("images/horizon_outer.png", cv2.IMREAD_UNCHANGED)
horiz_roll = cv2.imread("images/horizon_roll.png", cv2.IMREAD_UNCHANGED)
horiz_pitch = cv2.imread("images/horizon_pitch.png", cv2.IMREAD_UNCHANGED)

horiz_pitch = cv2.resize(horiz_pitch, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
horiz_roll = cv2.resize(horiz_roll, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
horiz_base = cv2.resize(horiz_base, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
horiz_outer = cv2.resize(horiz_outer, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)

alt_base = cv2.imread("images/altimeter_base.png")
alt_base = cv2.resize(alt_base, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
altitude = 15555
point_10km = cv2.imread("images/10km_pointer.PNG", cv2.IMREAD_UNCHANGED) 
point_10km = cv2.resize(point_10km, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
point_1km = cv2.imread("images/1km_pointer.PNG")
point_100m = cv2.imread("images/100m_pointer.PNG")

height, width = alt_base.shape[:2]
middle = [height/2, width/2]
roll_alt = 30
rot = cv2.getRotationMatrix2D(middle, roll_alt, 1)
inter1 = cv2.warpAffine(point_10km, rot, [width, height])

alpha_channel = inter1[:, :, 3] / 255 
overlay_colors = inter1[:, :, :3]
alpha_mask = np.dstack((alpha_channel, alpha_channel, alpha_channel))
result = alt_base * (1 - alpha_mask) + overlay_colors * alpha_mask

img = horizon.get(0,0, 1, horiz_base, horiz_outer, horiz_roll, horiz_pitch)

# Creates overlay for GUI
first_column = [
    [sg.Image(filename="", key='-3DPLOT-')],
    [sg.HorizontalSeparator()],
    [sg.Image(filename="", key='-2DPLOT-')]
]

second_column = [
    [sg.Image(filename="", key='-HORIZON-')],
    [sg.HorizontalSeparator()],
    [sg.Image(filename="", key='-ALTIMETER-')]
]

third_column = [
    [sg.Image(filename="", key ='-AIRSPEED-')],
    [sg.HorizontalSeparator()],
    [sg.Image(filename="", key='-FLIGHTDATA-')]
]
    

layout = [
    [
        sg.Column(first_column),
        sg.VSeparator(),
        sg.Column(second_column),
        sg.VSeparator(),
        sg.Column(third_column)
    ]
]

# Creates GUI window
window = sg.Window("Viewer", layout, resizable=True)
event, values = window.read(timeout=200)

# Loading load screen and plane image for GPS
load_image = cv2.imread('images/dumbshit.jpg')
plane_image = plt.imread('images/plane.png')

# Converts image to bytes
imgbytes = cv2.imencode(".png", load_image)[1].tobytes()
window["-3DPLOT-"].update(data=imgbytes)
window["-2DPLOT-"].update(data=imgbytes)
window["-HORIZON-"].update(data=imgbytes)
window["-ALTIMETER-"].update(data=imgbytes)
window["-AIRSPEED-"].update(data=imgbytes)
window["-FLIGHTDATA-"].update(data=imgbytes)

# Creates original state and control vector
state = np.array([10, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])
control_vec = np.array([0,0,0,0])
delta_t = 0.1


while True:
    event, values = window.read(timeout = 100)
    if event == sg.WIN_CLOSED:
        break
    
    # Updates dynamics and control 
    state = dynamics.step(state, control_vec, delta_t)
    control_vec = control.update(control_vec)
    
    plotter2D.GPS_Plotter(state, plane_image)
    plotter3D.vector_3D_plotter(state)
    
    pitch = state[10] * 10
    roll = state[9] * 10
    
    attitude = horizon.get(roll,-pitch, 1, horiz_base, horiz_outer, horiz_roll, horiz_pitch)
    img_horizon = cv2.imencode(".png", attitude)[1].tobytes()
    plot_3D = cv2.imread('images/vector3DPlot.jpg')
    img_3D_plotter = cv2.imencode(".png", plot_3D)[1].tobytes()
    plot_2D = cv2.imread('images/GPSPlot.jpg')
    img_2D_plotter = cv2.imencode(".png", plot_2D)[1].tobytes()
    
    window["-HORIZON-"].update(data=img_horizon)
    window["-3DPLOT-"].update(data=img_3D_plotter)
    window["-2DPLOT-"].update(data=img_2D_plotter)


window.close()

In [19]:
window.close()